Import required Libraries

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import errno
import os
import shutil

Set Data, Training and Test paths

In [ ]:
path = 'D://CVDL//Project//Caltech101'
train_path = 'D://CVDL//Project//train//'
test_path = 'D://CVDL//Project//test//'
os.mkdir(train_path)
os.mkdir(test_path)

Split data to train and test

In [ ]:
k = -1
os.chdir(path)
for root, dirs, files in os.walk(path, topdown=True):
    if(k==-1):
        dirrs = dirs
        k=0
    else:        
        test_len = round(len(files)*90/100)
        os.mkdir(test_path+ dirrs[k])
        os.mkdir(train_path+ dirrs[k])
            
        for i in range(0,round(len(files)*90/100)):
            shutil.copy(path+'//'+dirrs[k]+'//'+files[i],train_path+dirrs[k])
            i = i+1                    

        for i in range(round(len(files)*90/100),len(files)-1):  
            shutil.copy(path+'//'+dirrs[k]+'//'+files[i],test_path+dirrs[k])
            i = i+1
            
        k = k+1
        if(k>len(dirrs)-1):
            break


Read train and test images using ImageDataGenerator

In [ ]:
def readBatch():
    
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=7811,
        class_mode='categorical')
    print("please wait")
    k=0
    for i in train_generator:
        train_images,train_labels = i
        break
        
    test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    test_generator = test_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=1000,
        class_mode='categorical')
    print("please wait")
    for i in test_generator:
        test_images,test_labels = i
        break
    return train_images,train_labels,test_images,test_labels

Define the model

In [ ]:
def buildModel(train_images,train_labels):
    model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())

    model.add(Dense(101, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

    model.fit(train_images, train_labels, batch_size=32, epochs=1)
    #score = model.evaluate(x_test, y_test, batch_size=32)
    return model


Lets begin!

In [ ]:
train_images,train_labels,test_images,test_labels = readBatch();

#     from keras.models import load_model
#     model = load_model('modelfold09.h5')
#     correct = 0
#     for i in range(0,len(train_labels)):
#         if(train_labels[i] == model.predict(test_images[i])):
#             correct=correct+1
#     print("accuracy :"+ correct/len(train_labels))



Build the model

In [ ]:
model = buildModel(train_images,train_labels)

Get the score

In [ ]:
print("score"+str(model.evaluate(test_images,test_labels)))